#### 1. Imports and Set-up


In [ ]:
import pandas as pd
import numpy as np
import os

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, r2_score, mean_squared_error
from sklearn.preprocessing import OrdinalEncoder

import xgboost as xgb
import optuna

import matplotlib.pyplot as plt
import seaborn as sns

import mlflow
import mlflow.sklearn

pd.set_option("display.max_columns", None)
sns.set_style("whitegrid")

tracking_uri = "../logs/mlruns"
os.makedirs(os.path.join(tracking_uri, ".trash"), exist_ok=True)

mlflow.set_tracking_uri(tracking_uri)
mlflow.set_experiment("house_price_prediction")

#### 2. Load and prep data


In [ ]:
import sys
import os
from pathlib import Path
import yaml


# Adjust the path to your project root folder
project_root = os.path.abspath(
    os.path.join("..")
)  # from notebooks/ up one level

if project_root not in sys.path:
    sys.path.insert(0, project_root)

from src.data_loading.data_loading.data_loader import load_data_from_json
from src.data_loading.preprocessing.preprocessing import preprocess_df
from src.data_loading.preprocessing.imputation import impute_missing_values


# go two levels up from notebook dir -> project root
ROOT = (
    Path(__file__).resolve().parents[2]
    if "__file__" in globals()
    else Path.cwd().parents[1]
)
CONFIG_PATH = (
    ROOT
    / "house_price_prediction_project"
    / "config"
    / "preprocessing_config.yaml"
)

with open(CONFIG_PATH) as f:
    CONFIG = yaml.safe_load(f)

df_raw = load_data_from_json("../data/parsed_json/*.json")
df_clean = preprocess_df(
    df_raw,
    drop_raw=CONFIG["preprocessing"]["drop_raw"],
    numeric_cols=CONFIG["preprocessing"]["numeric_cols"],
)
df_clean = impute_missing_values(
    df_clean, CONFIG["preprocessing"]["imputation"]
)
# Drop price_num NaNs for the training of the model
df_clean = df_clean[df_clean["price_num"].notna()]
df_clean.drop(columns=["living_area"], inplace=True)

# df_clean = df_clean[:100] 
df = df_clean.copy()

In [ ]:
from collections import Counter

# Path to your house_pages.txt
file_path = (
    ROOT
    / "house_price_prediction_project"
    / "config"
    / "house_pages_scraped.txt"
)

# Read all URLs
with open(file_path, "r") as f:
    urls = f.read().splitlines()

# ✅ Count koop/amsterdam
count_amsterdam = sum(
    1 for url in urls if "koop" in url.lower() and "amsterdam" in url.lower()
)
print(f"Number of koop/amsterdam listings: {count_amsterdam}")


In [ ]:
df.isna().sum()

In [ ]:
df.columns

In [ ]:
from src.features.data_prep_for_modelling.data_preparation import prepare_data

FEATURES_CONFIG_PATH = (
    ROOT / "house_price_prediction_project" / "config" / "model_config.yaml"
)

# Scaled features (applies scaling according to YAML)
X_train_scaled, X_test_scaled, y_train, y_test, X_val, y_val, scaler, _ = prepare_data(
    df,
    config_path=FEATURES_CONFIG_PATH,
    model_name="linear_regression",  # uses the unified YAML key
    use_extended_features=False,       # set True if you want extended features
    cv=False
)

#### 4. Linear Regression


In [ ]:
from src.model.evaluate import ModelEvaluator
from src.model.mlflow_logger import MLFlowLogger

evaluator = ModelEvaluator()
logger = MLFlowLogger()

lr_model = LinearRegression()

# Evaluate
trained_lr, y_train_pred, y_val_pred, y_test_pred, lr_results = evaluator.evaluate(
    model=lr_model,
    X_train=X_train_scaled,
    y_train=y_train,
    X_test=X_test_scaled,
    y_test=y_test,
    model_params={},   
    fit_params={},     
    use_xgb_train=False
)

# Log the model and results
logger.log_model(trained_lr, "LinearRegression", lr_results)

#### 5. Random Forest Regression


In [ ]:
from src.features.feature_engineering.encoding import encode_energy_label

X_train, X_test, y_train, y_test, scaler, X_val, y_val, _ = prepare_data(
    df_clean,
    config_path=FEATURES_CONFIG_PATH, 
    model_name="random_forest",
    use_extended_features=False,     
    cv=False 
)

In [ ]:
X_train.energy_label_encoded.unique()

In [ ]:
rf_model = RandomForestRegressor()

trained_rf, y_train_pred, y_val_pred, y_test_pred, rf_results = evaluator.evaluate(
    model=rf_model,
    X_train=X_train,
    y_train=y_train,
    X_test=X_test,
    y_test=y_test,
    model_params={},  
    fit_params={},    
    use_xgb_train=False
)
logger.log_model(trained_rf, "RandomForestRegression", rf_results)

#### 6. XGBoost model


In [ ]:
from src.model.utils import load_model_config_and_search_space

X_train, X_test, y_train, y_test, X_val, y_val, scaler, _ = prepare_data(
    df_clean, config_path=FEATURES_CONFIG_PATH, model_name="xgboost", 
    use_extended_features=False, cv=False
)

MODEL_CONFIG_PATH = (
    ROOT / "house_price_prediction_project" / "config" / "model_config.yaml"
)

model_params, fit_params, _ = load_model_config_and_search_space(
    MODEL_CONFIG_PATH, model_name="xgboost"
)
fit_params_safe = fit_params.copy()
n_estimators = fit_params_safe.pop("n_estimators", 100)  

xgb_model = xgb.XGBRegressor(
    n_estimators=n_estimators,
    **model_params
)

trained_xgb, y_train_pred, y_val_pred, y_test_pred, xgb_results = evaluator.evaluate(
    xgb_model,
    X_train,
    y_train,
    X_test=X_test,
    y_test=y_test,
    fit_params=fit_params_safe, 
    use_xgb_train=False,
    X_val=X_val,
    y_val=y_val,
)
logger.log_model(trained_xgb, "XGBoostRegression", xgb_results)

#### 7. XGBoost with early stopping and more tuning


In [ ]:
X_train, X_test, y_train, y_test, X_val, y_val, scaler, _ = prepare_data(
    df_clean,
    config_path=FEATURES_CONFIG_PATH,
    model_name="xgboost_early_stopping",
    use_extended_features=False,
    cv=False,
)

xgb_model_params, xgb_fit_params, _ = load_model_config_and_search_space(
    MODEL_CONFIG_PATH, "xgboost_early_stopping"
)

xgb_model = xgb.XGBRegressor(**xgb_model_params)


trained_xgb, y_train_pred, y_val_pred, y_test_pred, xgb_results = evaluator.evaluate(
    None,
    X_train,
    y_train,
    X_test,
    y_test,
    X_val=X_val,
    y_val=y_val,
    fit_params=xgb_fit_params,
    model_params=xgb_model_params,
    use_xgb_train=True,  
)

logger.log_model(
    trained_xgb, "xgb_with_early_stopping", xgb_results, use_xgb_train=True
)

In [ ]:
results = {
    "LinearRegression": {
        'train_rmse': 234668.5789291843, 'test_rmse': 247683.6384448279,
        'train_mae': 133680.6268753051, 'test_mae': 135313.49701588583,
        'train_r2': 0.7993796779769898, 'test_r2': 0.7353680437834911,
        'train_mape': 21.82881987145939, 'test_mape': 21.525239863141383
    },
    "RandomForestRegression": {
        'train_rmse': 62941.14501995167, 'test_rmse': 231591.7199414145,
        'train_mae': 24255.64677922498, 'test_mae': 78288.51394001841,
        'train_r2': 0.9855677409706858, 'test_r2': 0.7686371071859328,
        'train_mape': 3.0138612062252124, 'test_mape': 8.327634305698103
    },
    "XGBoostRegression": {
        'train_rmse': 26988.016901703057, 'test_rmse': 228128.62098346377,
        'train_mae': 19190.51111202007, 'test_mae': 76143.21268168604,
        'train_r2': 0.9973465739818214, 'test_r2': 0.7755047274183167,
        'train_mape': 3.390482022105623, 'test_mape': 8.436821278014358
    },
    "xgb_with_early_stopping": {
        'train_rmse': 29500.180435952334, 'val_rmse': 170577.2863619771, 'test_rmse': 227379.80568905,
        'train_mae': 21063.793919340093, 'val_mae': 67491.76598837209, 'test_mae': 77027.80154433139,
        'train_r2': 0.9968117063387806, 'val_r2': 0.8978210876957955, 'test_r2': 0.7769760868294767,
        'train_mape': 3.7200625742427404, 'val_mape': 8.099211280295455, 'test_mape': 8.532328967753548
    }
}


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

sns.set(style="whitegrid", font_scale=1.1)

df_results = pd.DataFrame(results).T
df_results = df_results.rename(index={'xgb_with_early_stopping': 'XGBoost (Early Stop)'})
models = np.arange(len(df_results))
bar_width = 0.35
colors = ['skyblue', 'orange']  # train vs test

# Determine max for RMSE & MAE to share same scale
rmse_mae_max = df_results[['train_rmse', 'test_rmse', 'train_mae', 'test_mae']].max().max() * 1.1

fig, axes = plt.subplots(3, 1, figsize=(12, 14))

# ---------- RMSE ----------
rmse = df_results[['train_rmse', 'test_rmse']].rename(columns={'train_rmse':'Train RMSE (€)','test_rmse':'Test RMSE (€)'})
for i, col in enumerate(rmse.columns):
    axes[0].barh(models + i*bar_width - bar_width/2, rmse[col].values, height=bar_width, color=colors[i], label=col)

axes[0].set_yticks(models)
axes[0].set_yticklabels(df_results.index)
axes[0].invert_yaxis()
axes[0].set_xlabel("Error (€)")
axes[0].set_title("Model Comparison: RMSE")
axes[0].set_xlim(0, rmse_mae_max)

# Data labels
for i, col in enumerate(rmse.columns):
    for j, val in enumerate(rmse[col].values):
        axes[0].text(val + rmse_mae_max*0.01, j + i*bar_width - bar_width/2, f"{val:,.0f}", va='center', fontsize=9)

axes[0].legend(loc='upper left', bbox_to_anchor=(1,1))
# ---------- MAE ----------
mae = df_results[['train_mae', 'test_mae']].rename(columns={'train_mae':'Train MAE (€)','test_mae':'Test MAE (€)'})
for i, col in enumerate(mae.columns):
    axes[1].barh(models + i*bar_width - bar_width/2, mae[col].values, height=bar_width, color=colors[i], label=col)

axes[1].set_yticks(models)
axes[1].set_yticklabels(df_results.index)
axes[1].invert_yaxis()
axes[1].set_xlabel("Error (€)")
axes[1].set_title("Model Comparison: MAE")
axes[1].set_xlim(0, rmse_mae_max)  # same scale as RMSE

# Data labels
for i, col in enumerate(mae.columns):
    for j, val in enumerate(mae[col].values):
        axes[1].text(val + rmse_mae_max*0.01, j + i*bar_width - bar_width/2, f"{val:,.0f}", va='center', fontsize=9)

axes[1].legend(loc='upper left', bbox_to_anchor=(1,1))

# ---------- MAPE ----------
mape = df_results[['train_mape', 'test_mape']].rename(columns={'train_mape':'Train MAPE (%)','test_mape':'Test MAPE (%)'})
for i, col in enumerate(mape.columns):
    axes[2].barh(models + i*bar_width - bar_width/2, mape[col].values, height=bar_width, color=colors[i], label=col)

axes[2].set_yticks(models)
axes[2].set_yticklabels(df_results.index)
axes[2].invert_yaxis()
axes[2].set_xlabel("MAPE (%)")
axes[2].set_title("Model Comparison: MAPE")
axes[2].set_xlim(0, max(mape.max().max() * 1.1, 10))

# Data labels
for i, col in enumerate(mape.columns):
    for j, val in enumerate(mape[col].values):
        axes[2].text(val + 0.2, j + i*bar_width - bar_width/2, f"{val:.1f}%", va='center', fontsize=9)

axes[2].legend(loc='upper left', bbox_to_anchor=(1,1))

plt.tight_layout()
plt.show()


#### 7. Compare models using MLflow


In [ ]:
from src.model.evaluate import ModelEvaluator
from src.model.mlflow_logger import MLFlowLogger

evaluator = ModelEvaluator()
logger = MLFlowLogger()

experiment_name = "house_price_prediction"

experiment = mlflow.get_experiment_by_name(experiment_name)
experiment_id = experiment.experiment_id


runs_df = mlflow.search_runs(experiment_ids=[experiment_id])

In [ ]:
metrics_of_interest = [
    # Original scale
    "metrics.train_rmse",
    "metrics.test_rmse",
    "metrics.train_mae",
    "metrics.test_mae",
    "metrics.train_r2",
    "metrics.test_r2",
    "metrics.train_mape",
    "metrics.test_mape",
    # "metrics.train_huber",
    # "metrics.test_huber",
]
comparison_df = runs_df[
    ["run_id", "tags.mlflow.runName"] + metrics_of_interest
]

comparison_df.sort_values("metrics.test_r2", ascending=False, inplace=True)
comparison_df

In [ ]:
# Assuming comparison_df has a column like "tags.Mlflow.runName"
selected_models = [
    "XGB_Optuna_LogTransformed_feature_eng",
    "RF_LogTransform_Optuna_feature_eng"
]

filtered_df = comparison_df[comparison_df["tags.mlflow.runName"].isin(selected_models)]

metrics_df = filtered_df[["tags.mlflow.runName"] + metrics_of_interest]


metrics_df


In [ ]:
best_model = comparison_df.sort_values(
    "metrics.test_r2", ascending=False
).iloc[0]
print("Best model based on test R²:")
print(best_model)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# --- Combined baseline + tuned ---
data = [
    {"run_name": "RF_Baseline", "train_rmse": 97000, "test_rmse": 265000, "train_mae": 31000, "test_mae": 89000,
     "train_r2": 0.960, "test_r2": 0.700, "train_mape": 3.8, "test_mape": 9.0},
    {"run_name": "RF_LogTransform_Optuna_feature_eng", "train_rmse": 90502.66175481866, "test_rmse": 252430.01455217763,
     "train_mae": 29120.13411482258, "test_mae": 77855.07702928812, "train_r2": 0.9699923714149887,
     "test_r2": 0.7251285494583974, "train_mape": 3.4799216014008447, "test_mape": 8.382640083172193},
    {"run_name": "XGB_Baseline", "train_rmse": 95000, "test_rmse": 240000, "train_mae": 52000, "test_mae": 85000,
     "train_r2": 0.965, "test_r2": 0.780, "train_mape": 7.2, "test_mape": 9.1},
    {"run_name": "XGB_Optuna_LogTransformed_feature_eng", "train_rmse": 87195.54760564862, "test_rmse": 211171.78653958422,
     "train_mae": 45664.83483626995, "test_mae": 72963.94520348837, "train_r2": 0.972145357425832,
     "test_r2": 0.8076379317581717, "train_mape": 6.45700927169167, "test_mape": 8.460645105316997}
]

df_results = pd.DataFrame(data).set_index("run_name")

# Optional: nicer labels
df_results.index = ["RF Baseline", "RF Optuna + FE", "XGB Baseline", "XGB Optuna + FE"]

# --- Plotting ---
sns.set(style="whitegrid", font_scale=1.1)
models = np.arange(len(df_results))
bar_width = 0.35
colors = ['skyblue', 'orange']
rmse_mae_max = df_results[['train_rmse','test_rmse','train_mae','test_mae']].max().max() * 1.1

fig, axes = plt.subplots(3, 1, figsize=(12, 16))

# ---------- RMSE ----------
rmse = df_results[['train_rmse','test_rmse']].rename(columns={'train_rmse':'Train RMSE (€)', 'test_rmse':'Test RMSE (€)'})
for i, col in enumerate(rmse.columns):
    axes[0].barh(models + i*bar_width - bar_width/2, rmse[col].values, height=bar_width, color=colors[i], label=col)

axes[0].set_yticks(models)
axes[0].set_yticklabels(df_results.index)
axes[0].invert_yaxis()
axes[0].set_xlabel("Error (€)")
axes[0].set_title("Baseline Model Comparison with the Improved Models: RMSE")
axes[0].set_xlim(0, rmse_mae_max)

# metric labels
for i, col in enumerate(rmse.columns):
    for j, val in enumerate(rmse[col].values):
        axes[0].text(val + rmse_mae_max*0.01, j + i*bar_width - bar_width/2, f"{val:,.0f}", va='center', fontsize=9)

axes[0].legend(loc='upper left', bbox_to_anchor=(1,1))

# ---------- MAE ----------
mae = df_results[['train_mae','test_mae']].rename(columns={'train_mae':'Train MAE (€)','test_mae':'Test MAE (€)'})
for i, col in enumerate(mae.columns):
    axes[1].barh(models + i*bar_width - bar_width/2, mae[col].values, height=bar_width, color=colors[i], label=col)

axes[1].set_yticks(models)
axes[1].set_yticklabels(df_results.index)
axes[1].invert_yaxis()
axes[1].set_xlabel("Error (€)")
axes[1].set_title("Baseline Model Comparison with the Improved Models: MAE")
axes[1].set_xlim(0, rmse_mae_max)

# metric labels
for i, col in enumerate(mae.columns):
    for j, val in enumerate(mae[col].values):
        axes[1].text(val + rmse_mae_max*0.01, j + i*bar_width - bar_width/2, f"{val:,.0f}", va='center', fontsize=9)

axes[1].legend(loc='upper left', bbox_to_anchor=(1,1))

# ---------- MAPE ----------
mape = df_results[['train_mape','test_mape']].rename(columns={'train_mape':'Train MAPE (%)','test_mape':'Test MAPE (%)'})
for i, col in enumerate(mape.columns):
    axes[2].barh(models + i*bar_width - bar_width/2, mape[col].values, height=bar_width, color=colors[i], label=col)

axes[2].set_yticks(models)
axes[2].set_yticklabels(df_results.index)
axes[2].invert_yaxis()
axes[2].set_xlabel("MAPE (%)")
axes[2].set_title("Baseline Model Comparison with the Improved Models: MAPE")
axes[2].set_xlim(0, max(mape.max().max() * 1.1, 10))

# metric labels
for i, col in enumerate(mape.columns):
    for j, val in enumerate(mape[col].values):
        axes[2].text(val + 0.2, j + i*bar_width - bar_width/2, f"{val:.1f}%", va='center', fontsize=9)

axes[2].legend(loc='upper left', bbox_to_anchor=(1,1))

plt.tight_layout()
plt.show()


#### 8. Hyperparameter tuning with Optuna


In [ ]:
from functools import partial
from src.model.objectives_optuna import unified_objective

FEATURES_AND_MODEL_CONFIG_PATH = (
    ROOT
    / "house_price_prediction_project"
    / "config"
    / "model_config.yaml"
)

# XGBoost study
sampler = optuna.samplers.TPESampler(seed=42)
pruner = optuna.pruners.MedianPruner(n_warmup_steps=10)
study_xgb = optuna.create_study(direction="minimize", sampler=sampler, pruner=pruner)
objective_xgb_partial = partial(
    unified_objective,
    model_name="xgboost_early_stopping",
    df=df_clean,
    features_config=FEATURES_AND_MODEL_CONFIG_PATH,
    model_config=FEATURES_AND_MODEL_CONFIG_PATH,
    use_extended_features=False,
)
study_xgb.optimize(objective_xgb_partial, n_trials=10)

print("Best XGBoost params:", study_xgb.best_params)
print("Best XGBoost Test RMSE:", study_xgb.best_value)

# # RandomForest study
# study_rf = optuna.create_study(direction="minimize")
# objective_rf_partial = partial(
#     unified_objective,
#     model_name="random_forest_optuna",
#     df=df_clean,
#     features_config=FEATURES_AND_MODEL_CONFIG_PATH,
#     model_config=FEATURES_AND_MODEL_CONFIG_PATH,
#     use_extended_features=False,
# )
# study_rf.optimize(objective_rf_partial, n_trials=50)

# print("Best RF params:", study_rf.best_params)
# print("Best RF Test RMSE:", study_rf.best_value)

#### 9. RF and Xgboost with best parameters


In [ ]:
# best_rf = RandomForestRegressor(**study_rf.best_params)
# trained_rf, y_train_pred, y_val_pred, y_test_pred, results_rf = evaluator.evaluate(
#     model=best_rf,
#     X_train=X_train,
#     y_train=y_train,
#     X_test=X_test,
#     y_test=y_test,
#     X_val=X_val,
#     y_val=y_val,
#     fit_params={},
# )

# logger.log_model(trained_rf, "RF_Optuna", results_rf)

In [ ]:
X_train, X_test, y_train, y_test, X_val, y_val, _, _ = prepare_data(
    df_clean,
    config_path=FEATURES_AND_MODEL_CONFIG_PATH,
    model_name="xgboost_early_stopping",
    use_extended_features=False,
    cv=False
)


xgb_model_params, xgb_fit_params, _ = load_model_config_and_search_space(
    MODEL_CONFIG_PATH, "xgboost_early_stopping"
)

# xgb_model = xgb.XGBRegressor(**study_xgb.best_params)

trained_xgb, _, _, _, xgb_results = evaluator.evaluate(
    model=None,  
    X_train=X_train,
    y_train=y_train,
    X_test=X_test,
    y_test=y_test,
    X_val=X_val,
    y_val=y_val,
    model_params=study_xgb.best_params,
    fit_params=xgb_fit_params,
    use_xgb_train=True,
)

logger.log_model(
    trained_xgb, "xgb_with_early_stopping_optuna", xgb_results, use_xgb_train=True
)


#### 10. Let's see how outliers skew RMSE


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Plot target distribution
sns.boxplot(y=y_test)
plt.show()

# Optional: scatter of predictions vs true values
plt.scatter(y_test, y_test_pred, alpha=0.5)
plt.xlabel("True")
plt.ylabel("Predicted")
plt.show()

In [ ]:
# Compute residuals
residuals = y_test - y_test_pred

# Summary stats
print("Residuals summary:")
print("Min:", np.min(residuals))
print("Max:", np.max(residuals))
print("Median:", np.median(residuals))
print("Mean:", np.mean(residuals))
print("Std:", np.std(residuals))

# Plot histogram
plt.hist(residuals, bins=50)
plt.title("Residuals Distribution")
plt.xlabel("Residual")
plt.ylabel("Count")
plt.show()

In [ ]:
from sklearn.metrics import mean_absolute_error

train_mae = mean_absolute_error(y_train, y_train_pred)
test_mae = mean_absolute_error(y_test, y_test_pred)

print(f"Train MAE: {train_mae:.2f}")
print(f"Test MAE:  {test_mae:.2f}")

So outliers are skewing the RMSE statistic quite heavily. Hence, I will log transform the target and dot he same analysis.


In [ ]:
# evaluator = ModelEvaluator(
#     target_transform=np.log1p,
#     inverse_transform=np.expm1
# )
# best_rf = RandomForestRegressor(**study_rf.best_params)

# trained_rf, y_train_pred, y_val_pred, y_test_pred, results = evaluator.evaluate(
#     model=best_rf,
#     X_train=X_train,
#     y_train=y_train,
#     X_test=X_test,
#     y_test=y_test,
#     use_xgb_train=False  
# )
# logger.log_model(trained_rf, "RF_LogTransform_Evaluator", results)


In [ ]:
from xgboost import XGBRegressor

# Initialize XGBoost with best params (from previous Optuna run)
best_xgb = XGBRegressor(**study_xgb.best_params)

# Use log transform
evaluator = ModelEvaluator(
    target_transform=np.log1p,
    inverse_transform=np.expm1
)

# Evaluate model
trained_xgb, y_train_pred, y_val_pred, y_test_pred, results = evaluator.evaluate(
    model=None,
    X_train=X_train,
    y_train=y_train,
    X_val=X_val,       
    y_val=y_val,     
    X_test=X_test,
    y_test=y_test,
    use_xgb_train=True,
    model_params=study_xgb.best_params
)

# Log the trained model
logger.log_model(
    trained_xgb, 
    "XGB_LogTransform_Evaluator", 
    results, 
    use_xgb_train=True
)


In [ ]:
# Compute residuals
residuals = y_test - y_test_pred

# Define extreme outliers: e.g., top 5% of absolute residuals
threshold = np.percentile(np.abs(residuals), 95)
outliers_mask = np.abs(residuals) >= threshold

plt.figure(figsize=(8, 6))

# Plot non-outliers
plt.scatter(
    y_test[~outliers_mask],
    y_test_pred[~outliers_mask],
    alpha=0.5,
    label="Normal listings",
)

# Highlight extreme residuals
plt.scatter(
    y_test[outliers_mask],
    y_test_pred[outliers_mask],
    color="red",
    label="Extreme listings",
)

# Diagonal line (perfect prediction)
max_val = max(y_test.max(), y_test_pred.max())
plt.plot(
    [0, max_val],
    [0, max_val],
    color="black",
    linestyle="--",
    label="Perfect prediction",
)

plt.xlabel("Actual Price (€)")
plt.ylabel("Predicted Price (€)")
plt.title("Predicted vs Actual Prices with Extreme Listings Highlighted")
plt.legend()
plt.show()

This is a clear visualization to see how predictions behave across the entire range and highlight the extreme listings that inflate RMSE. Large RMSE is not a deal breaker since:

**Statistical justification**
Skewed distribution: My dataset has a few extremely expensive houses that are far from the mean. RMSE is sensitive to large errors because it squares residuals, so these few points dominate the metric.

MAE is more robust: By reporting MAE alongside RMSE, I show the typical prediction error for most listings, which is a fairer assessment of model performance.

Log-transform mitigates skew: Training on log1p(y) reduces the influence of outliers and stabilizes variance, producing a more reliable model for the bulk of the data.

**Practical/business justification**

The extreme listings (multi-million € homes) are rare. The model performs well on 99% of listings, which is what matters for most users or business decisions.

Trying to perfectly predict the top 1–5% of luxury listings would:

Require specialized models or additional data

Complicate the pipeline

Increase overfitting risk

Reporting MAE and residual distributions communicates clearly that errors on extreme listings exist, but are expected and do not invalidate the model.


#### 11. New approach and moving with optuna


In [ ]:
# XGBoost with log-transform
sampler = optuna.samplers.TPESampler(seed=42)
pruner = optuna.pruners.MedianPruner(n_warmup_steps=10)
study_xgb = optuna.create_study(direction="minimize", sampler=sampler, pruner=pruner)

objective_xgb_partial = partial(
    unified_objective,
    model_name="xgboost_early_stopping",
    df=df_clean,
    features_config=FEATURES_AND_MODEL_CONFIG_PATH,
    model_config=FEATURES_AND_MODEL_CONFIG_PATH,
    use_log=True,  
    n_splits=5,
    use_extended_features=False,
)

study_xgb.optimize(objective_xgb_partial, n_trials=30)

# Random Forest with log-transform
study_rf = optuna.create_study(direction="minimize")

objective_rf_partial = partial(
    unified_objective,
    model_name="random_forest_optuna",
    df=df_clean,
    features_config=FEATURES_AND_MODEL_CONFIG_PATH,
    model_config=FEATURES_AND_MODEL_CONFIG_PATH,
    use_log=True,  
    n_splits=5,
    use_extended_features=False,
)


study_rf.optimize(objective_rf_partial, n_trials=30)

In [ ]:
# Initialize evaluator with log-transform (same as used in Optuna)
evaluator = ModelEvaluator(
    target_transform=np.log1p,
    inverse_transform=np.expm1,
)

# --- Random Forest ---
best_rf = RandomForestRegressor(**study_rf.best_params)
trained_rf, y_train_pred, y_val_pred, y_test_pred, results_rf = evaluator.evaluate(
    model=best_rf,
    X_train=X_train,
    y_train=y_train,
    X_test=X_test,
    y_test=y_test,
    X_val=X_val,
    y_val=y_val,
    use_xgb_train=False,  # RF uses sklearn API
)
logger.log_model(trained_rf, "RF_Optuna_Log", results_rf, use_xgb_train=False)

# --- XGBoost ---
best_xgb_params = study_xgb.best_params
trained_xgb, y_train_pred, y_val_pred, y_test_pred, results_xgb = evaluator.evaluate(
    model=None,  # we pass params dict instead of a model instance
    X_train=X_train,
    y_train=y_train,
    X_test=X_test,
    y_test=y_test,
    X_val=X_val,
    y_val=y_val,
    use_xgb_train=True,  # XGBoost-specific training
    model_params=best_xgb_params,  # pass best hyperparams
    fit_params={"num_boost_round": 1000, "early_stopping_rounds": 50},  # you can tune these if needed
)
logger.log_model(trained_xgb, "XGB_Optuna_Log", results_xgb, use_xgb_train=True)


In [ ]:
experiment_name = "house_price_prediction"

experiment = mlflow.get_experiment_by_name(experiment_name)
experiment_id = experiment.experiment_id


runs_df = mlflow.search_runs(experiment_ids=[experiment_id])

runs_df['start_time_dt'] = pd.to_datetime(runs_df['start_time'], unit='ms')

# Filter runs between two dates
mask = (runs_df['start_time_dt'] >= '2025-09-18')

metrics_of_interest = [
    # Original scale
    "metrics.train_rmse",
    "metrics.test_rmse",
    "metrics.train_mae",
    "metrics.test_mae",
    "metrics.train_r2",
    "metrics.test_r2",
    "metrics.train_mape",
    "metrics.test_mape",
    
    # Log / transformed scale
    # "metrics.train_rmse_trans",
    # "metrics.test_rmse_trans",
    # "metrics.train_mae_trans",
    # "metrics.test_mae_trans",
    # "metrics.train_r2_trans",
    # "metrics.test_r2_trans",
    # "metrics.train_mape_trans",
    # "metrics.test_mape_trans",
]
comparison_df = runs_df[
    ["run_id", "tags.mlflow.runName"] + metrics_of_interest
]

comparison_df.sort_values("metrics.test_mae", ascending=True, inplace=True)
comparison_df = comparison_df[mask]
comparison_df

In [ ]:
comparison_df.sort_values("metrics.test_mae", ascending=True, inplace=True)
comparison_df = comparison_df[mask]
comparison_df

In [ ]:
best_model = comparison_df.sort_values(
    "metrics.test_mae", ascending=True
).iloc[0]
print("Best model based on test MAE:")
print(best_model)

#### 12. Extra feature engineering


In [ ]:
df_clean.columns

In [ ]:
FEATURES_AND_MODEL_CONFIG_PATH = (
    ROOT
    / "house_price_prediction_project"
    / "config"
    / "model_config.yaml"
)
# --- Prepare data for final modeling ---
X_train, X_test, y_train, y_test, X_val, y_val, scaler, feature_encoders = prepare_data(
    df=df_clean,
    config_path=FEATURES_AND_MODEL_CONFIG_PATH,
    model_name="xgboost_early_stopping",  
    use_extended_features=True,           
    cv=False                              
)


print("Train shape:", X_train.shape)
print("Validation shape:", X_val.shape if X_val is not None else None)
print("Test shape:", X_test.shape)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Select only numeric columns from X_train
X_numeric = X_train.select_dtypes(include="number")

# Correlation matrix
corr_matrix = X_numeric.corr()

# Heatmap
plt.figure(figsize=(12, 10))
sns.heatmap(corr_matrix, annot=True, fmt=".2f", cmap="coolwarm")
plt.title("Correlation matrix for numeric features (Train set)")
plt.show()

# Identify highly correlated pairs
high_corr = []
cols = corr_matrix.columns
for i in range(len(cols)):
    for j in range(i + 1, len(cols)):
        corr_val = corr_matrix.iloc[i, j]
        if abs(corr_val) > 0.9:
            high_corr.append((cols[i], cols[j], corr_val))

print("Highly correlated numeric pairs (|r|>0.9):")
for pair in high_corr:
    print(pair)


In [ ]:
cols_to_drop = [
    "size_num",
    # "living_area",
    # "nr_rooms",
    # "bathrooms",
    # "toilets",
    "num_facilities",
    # "external_storage_num",
]
X_train_final = X_train.copy()
X_test_final = X_test.copy()
X_val_final = X_val.copy()
X_train_final.drop(columns=cols_to_drop, inplace=True)
X_val_final.drop(columns=cols_to_drop, inplace=True)
X_test_final.drop(columns=cols_to_drop, inplace=True)

print("Train shape:", X_train_final.shape)
print("validation shape:", X_val_final.shape)
print("Test shape:", X_test_final.shape)


## 13. Baseline models after feature engineering


#### Baseline Random Forest


In [ ]:
#no need for validation set her
X_train_full = pd.concat([X_train_final, X_val_final], axis=0)
y_train_full = pd.concat([y_train, y_val], axis=0)

evaluator = ModelEvaluator(
    target_transform=np.log1p,     # log-transform target for training
    inverse_transform=np.expm1     # convert predictions back to original scale
)

rf_model = RandomForestRegressor()

trained_rf, y_train_pred, y_val_pred, y_test_pred, rf_results = evaluator.evaluate(
    model=rf_model,
    X_train=X_train_full,
    y_train=y_train_full,
    X_test=X_test_final,
    y_test=y_test,
    use_xgb_train=False, 

)
logger.log_model(trained_rf, "Random_Forest_Regression_feature_eng", rf_results,  use_xgb_train=False)

#### Baseline XGboost with early stopping


In [ ]:
MODEL_CONFIG_PATH = ROOT / "house_price_prediction_project" / "config" / "model_config.yaml"
model_params, fit_params, _ = load_model_config_and_search_space(MODEL_CONFIG_PATH, model_name="xgboost_early_stopping")

fit_params_safe = fit_params.copy()
n_estimators = fit_params_safe.pop("n_estimators", 100)  

xgb_model = xgb.XGBRegressor(
    n_estimators=n_estimators,
    **model_params
)


trained_xgb, y_train_pred, y_val_pred, y_test_pred, xgb_results = evaluator.evaluate(
    xgb_model,
    X_train_final,
    y_train,
    X_test_final,
    y_test,
    X_val=X_val_final,
    y_val=y_val,
    fit_params=fit_params,
    use_xgb_train=True  # ensures early stopping is used
)

# Log model
logger.log_model(trained_xgb, "XGBoostRegressionFeatureEng", xgb_results, use_xgb_train=True)


## 14. Optuna tuning after feature eng


In [ ]:
df_clean.columns

In [ ]:
# XGBoost with log-transform

from functools import partial
from src.model.objectives_optuna import unified_objective

FEATURES_AND_MODEL_CONFIG_PATH = (
    ROOT
    / "house_price_prediction_project"
    / "config"
    / "model_config.yaml"
)

sampler = optuna.samplers.TPESampler(seed=42)
pruner = optuna.pruners.MedianPruner(n_warmup_steps=10)
study_xgb = optuna.create_study(direction="minimize", sampler=sampler, pruner=pruner)

objective_xgb_partial = partial(
    unified_objective,
    model_name="xgboost_early_stopping_optuna_feature_eng",
    df=df_clean,
    features_config=FEATURES_AND_MODEL_CONFIG_PATH,
    model_config=FEATURES_AND_MODEL_CONFIG_PATH,
    use_log=True,  
    n_splits=5,
    use_extended_features=True
)
study_xgb.optimize(objective_xgb_partial, n_trials=100)

# # Random Forest with log-transform
# study_rf = optuna.create_study(direction="minimize")

# objective_rf_partial = partial(
#     unified_objective,
#     model_name="random_forest_optuna_feature_eng",
#     df=df_clean,
#     features_config=FEATURES_AND_MODEL_CONFIG_PATH,
#     model_config=FEATURES_AND_MODEL_CONFIG_PATH,
#     use_log=True,  
#     n_splits=5,
#     use_extended_features=True

# )

# study_rf.optimize(objective_rf_partial, n_trials=50)

In [ ]:
import matplotlib.pyplot as plt
import optuna

# Suppose you have your study object already
# study = optuna.load_study(study_name="your_study_name", storage="your_storage")

# For demonstration, let's assume you have a study object:
# study = your Optuna study from your XGBoost optimization

# Extract trial numbers and values
trial_numbers = [t.number for t in study_xgb.trials]
objective_values = [t.value for t in study_xgb.trials]

# Find the best trial
best_trial = study_xgb.best_trial
best_number = best_trial.number
best_value = best_trial.value

# Plot
plt.figure(figsize=(12, 6))
plt.scatter(trial_numbers, objective_values, color='blue', label='Trials')
plt.scatter(best_number, best_value, color='red', s=100, label='Best Trial')
plt.xlabel('Trial Number')
plt.ylabel('Objective Value')
plt.title('Optuna XGBoost Trials')
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import optuna


# Sort trials by number
trials = sorted(study_xgb.trials, key=lambda t: t.number)
trial_numbers = [t.number for t in trials]
objective_values = [t.value for t in trials]

# Compute best-so-far values
best_so_far = []
current_best = float('inf')  # minimizing objective
best_trial_index = 0
for i, value in enumerate(objective_values):
    if value < current_best:
        current_best = value
        best_trial_index = i
    best_so_far.append(current_best)

# Best trial info
best_trial = trials[best_trial_index]
best_params = best_trial.params
best_value = best_trial.value

# Plot
plt.figure(figsize=(14, 7))
plt.scatter(trial_numbers, objective_values, color='lightblue', label='Trial Values')
plt.plot(trial_numbers, best_so_far, color='red', linewidth=2, label='Best-So-Far')

# Highlight best trial
plt.scatter(best_trial.number, best_value, color='green', s=150, marker='*', label='Best Trial')

# Annotate best trial hyperparameters
param_text = "\n".join([f"{k}: {v}" for k, v in best_params.items()])
plt.annotate(f'Best Trial #{best_trial.number}\nValue: {best_value:.4f}\n{param_text}',
             xy=(best_trial.number, best_value),
             xytext=(best_trial.number + 0.5, best_value + 0.5),
             arrowprops=dict(facecolor='black', arrowstyle='->'),
             bbox=dict(boxstyle="round,pad=0.3", fc="yellow", alpha=0.3))

plt.xlabel('Trial Number')
plt.ylabel('Objective Value')
plt.title('Optuna XGBoost Optimization Convergence with Best Trial')
plt.legend()
plt.grid(True)
plt.show()


## 15. Best params run after feature eng


In [ ]:
# Initialize evaluator with log-transform if used
# evaluator = ModelEvaluator(target_transform=np.log1p, inverse_transform=np.expm1)

# # --- Random Forest ---
# best_rf = RandomForestRegressor(**study_rf.best_params)
# trained_rf, y_train_pred, y_val_pred, y_test_pred, results_rf = evaluator.evaluate(
#     model=best_rf,
#     X_train=X_train_final,
#     y_train=y_train,
#     X_test=X_test_final,
#     y_test=y_test,
#     X_val=X_val_final,
#     y_val=y_val,
#     use_xgb_train=False,
# )
# logger.log_model(trained_rf, "RF_LogTransform_Optuna_feature_eng", results_rf, use_xgb_train=False)

# --- XGBoost ---
best_xgb_params = study_xgb.best_params
trained_xgb, y_train_pred, y_val_pred, y_test_pred, results_xgb = evaluator.evaluate(
    model=None,  # not used in XGBoost.train
    X_train=X_train_final,
    y_train=y_train,
    X_test=X_test_final,
    y_test=y_test,
    X_val=X_val_final,
    y_val=y_val,
    use_xgb_train=True,
    model_params=best_xgb_params,  # <--- crucial
    fit_params={"num_boost_round": 1000, "early_stopping_rounds": 50},
)
logger.log_model(trained_xgb, "XGB_Optuna_LogTransformed_feature_eng", results_xgb, use_xgb_train=True)


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import xgboost as xgb

# --- Feature importance types ---
importance_types = ["weight", "gain", "cover"]

# Dictionary to hold sorted importance dataframes
importance_dfs = {}

for imp_type in importance_types:
    # Get raw importance dictionary
    raw_importance = trained_xgb.get_score(importance_type=imp_type)

    # Convert to DataFrame
    df_imp = pd.DataFrame.from_dict(raw_importance, orient="index", columns=[imp_type])
    df_imp.index.name = "feature"
    df_imp = df_imp.sort_values(by=imp_type, ascending=False)

    importance_dfs[imp_type] = df_imp

    # Print top features
    print(f"\nTop 10 features by {imp_type}:")
    print(df_imp.head(80))

    # Optional: plot top 20 features
    df_imp.head(80).plot.barh(figsize=(8,6), legend=False, title=f"Top 20 features by {imp_type}")
    plt.gca().invert_yaxis()
    plt.show()


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import xgboost as xgb
from sklearn.model_selection import KFold
import numpy as np

from src.model.cv_helpers import prepare_base_data
from src.features.feature_engineering import feature_engineering_cv as fe_cv

prepare_fold_features = fe_cv.prepare_fold_features

# Number of folds (should match your CV setup)
N_SPLITS = 5

# Prepare base data (features + target)
X_full, y_full = prepare_base_data(df_clean, FEATURES_AND_MODEL_CONFIG_PATH, "xgboost_early_stopping_optuna_feature_eng")

importance_types = ["weight", "gain", "cover"]
fold_importances = {imp_type: [] for imp_type in importance_types}

kf = KFold(n_splits=N_SPLITS, shuffle=True, random_state=42)

for fold, (train_idx, val_idx) in enumerate(kf.split(X_full), 1):
    X_train, X_val = X_full.iloc[train_idx].copy(), X_full.iloc[val_idx].copy()
    y_train, y_val = y_full.iloc[train_idx].copy(), y_full.iloc[val_idx].copy()
    
    # Prepare fold-wise features (ensure extended feature engineering matches training)
    X_train_fold, X_val_fold, _, _ = prepare_fold_features(X_train, X_val, use_extended_features=True)
    
    # Train XGBoost on this fold using best params
    dtrain = xgb.DMatrix(X_train_fold, label=np.log1p(y_train))
    dval = xgb.DMatrix(X_val_fold, label=np.log1p(y_val))
    
    model_fold = xgb.train(
        params=best_xgb_params,
        dtrain=dtrain,
        num_boost_round=1000,
        evals=[(dval, "validation")],
        early_stopping_rounds=50,
        verbose_eval=False
    )
    
    # Collect importance for each type
    for imp_type in importance_types:
        imp_dict = model_fold.get_score(importance_type=imp_type)
        df_imp = pd.DataFrame.from_dict(imp_dict, orient="index", columns=[imp_type])
        df_imp.index.name = "feature"
        fold_importances[imp_type].append(df_imp)

# --- Aggregate across folds ---
agg_importances = {}
for imp_type, dfs in fold_importances.items():
    # Combine all folds into a single dataframe
    df_all = pd.concat(dfs, axis=1).fillna(0)
    df_all["mean"] = df_all.mean(axis=1)
    df_all = df_all.sort_values(by="mean", ascending=False)
    agg_importances[imp_type] = df_all
    print(f"\nTop 10 features by mean {imp_type} across folds:")
    print(df_all["mean"].head(100))
    
    # Plot top 20
    df_all["mean"].head(200).plot.barh(figsize=(10,6), title=f"Top 20 features by mean {imp_type} across folds")
    plt.gca().invert_yaxis()
    plt.show()


#### SHAP

In [ ]:
import shap

fold_shap_values = []

for fold, (train_idx, val_idx) in enumerate(kf.split(X_full), 1):
    X_train, X_val = X_full.iloc[train_idx].copy(), X_full.iloc[val_idx].copy()
    y_train, y_val = y_full.iloc[train_idx].copy(), y_full.iloc[val_idx].copy()
    
    # Prepare fold-wise features
    X_train_fold, X_val_fold, _, _ = prepare_fold_features(X_train, X_val, use_extended_features=True)
    
    # Train XGBoost
    dtrain = xgb.DMatrix(X_train_fold, label=np.log1p(y_train))
    dval = xgb.DMatrix(X_val_fold, label=np.log1p(y_val))
    
    model_fold = xgb.train(
        params=best_xgb_params,
        dtrain=dtrain,
        num_boost_round=1000,
        evals=[(dval, "validation")],
        early_stopping_rounds=50,
        verbose_eval=False
    )
    
    # --- SHAP ---
    explainer = shap.Explainer(model_fold)
    shap_values = explainer(X_val_fold)
    
    # Store mean absolute SHAP values per feature for this fold
    fold_shap_values.append(pd.DataFrame({
        "feature": X_val_fold.columns,
        "mean_abs_shap": np.abs(shap_values.values).mean(axis=0)
    }))


In [ ]:
# Combine all folds
df_shap_all = pd.concat(fold_shap_values)

# Group by feature and compute mean across folds
agg_shap = df_shap_all.groupby("feature")["mean_abs_shap"].mean().sort_values(ascending=False)

print("\nTop 20 features by mean absolute SHAP value across folds:")
print(agg_shap.head(20))

# Plot
agg_shap.head(20).sort_values().plot.barh(figsize=(10,6), title="Top 20 features by SHAP value")
plt.show()


#### DONE

In [ ]:
experiment_name = "house_price_prediction"

experiment = mlflow.get_experiment_by_name(experiment_name)
experiment_id = experiment.experiment_id


runs_df = mlflow.search_runs(experiment_ids=[experiment_id])

metrics_of_interest = [
    "metrics.train_rmse",
    "metrics.test_rmse",
    "metrics.train_r2",
    "metrics.test_r2",
    "metrics.train_mae",
    "metrics.test_mae",
    "metrics.train_mape",
    "metrics.test_mape",
]
comparison_df = runs_df[
    ["run_id", "tags.mlflow.runName"] + metrics_of_interest
]

comparison_df.sort_values("metrics.test_mae", ascending=True, inplace=True)
comparison_df